# Task 1
Watch this week's mandatory video and answer the following question: Which six activation functions are discussed during the talk?

Sigmoid, tanh, ReLU, Maxout, ELU

In [ ]:
import numpy as np
np.random.seed(1)

# Task 2.1 Dataset reader (1P)
Implement a reader for the dataset files which returns the input vectors $\mathbf{x}$ and labels $y$ as numpy arrays. The shape and number of returned arrays is up to you.

In [ ]:
def filereader(textfile):
    #read file from disk into memory
    f = open(textfile)
    raw = f.read()
    f.close()
    
    entries = raw.split('\n')
    data = []
    for i in range(len(entries)):
        if(len(entries[i].split('\t')) > 2):
            review = entries[i].split('\t')[0]
            if(entries[i].split('\t')[1][-3] == 'N'):
                polarity = np.array(0, dtype='float_')
            else:
                polarity = np.array(1, dtype='float_')

            word_vektor = np.array(entries[i].split('\t')[2].split(' '), dtype='float_')
            word_vektor = np.append(word_vektor, 1)
            data.append([review, polarity, word_vektor])
        
    return data

In [ ]:
train_data = filereader('DATA/rt-polarity.train.vecs')
dev_data = filereader('DATA/rt-polarity.dev.vecs')
test_set = filereader('DATA/rt-polarity.test.vecs')

# Task 2.2 Numpy implementation (5P)

a) Implement the perceptron stated above only using numpy. Include a method which computes the square loss and the accuracy of the model, given a dataset and a weight vector `w`. (3P)

In [ ]:
def sigmoid(x, w):
    return 1/(1+np.exp(-1*np.dot(x, w).astype('float_')))

def dsigmoid(x, w):
    return sigmoid(x, w)*(1-sigmoid(x, w))

In [ ]:
def update(train_data, w, batch_size, rate):
    #rate = 0.01
    l = len(train_data)
    np.random.shuffle(train_data)
    #batch_size = np.random.randint(l)
    
    batches = [train_data[i:i + batch_size] for i in range(0, l, batch_size)]
    for mini_batch in batches:
        mini_batch = [(entry[2].reshape(1,101), entry[1]) for entry in mini_batch]
    
        w = w - (rate/batch_size)*sum((sigmoid(x_y[0],w)-x_y[1])*dsigmoid(x_y[0],w)*x_y[0].transpose()
                for x_y in mini_batch)
    
    return w

In [ ]:
def get_loss(data, w):
    xy_data = [(entry[2], entry[1]) for entry in data]
    return int(sum((sigmoid(x_y[0],w)-x_y[1])**2
            for x_y in xy_data))
    

In [ ]:
def accuracy(data, w):
    P = sum(1 for entry in data if entry[1]==1)
    N = sum(1 for entry in data if entry[1]==0)
    TP = sum(1 for entry in data if 
              sigmoid(entry[2],w) >= 0.5 and entry[1]==1)
    TN = sum(1 for entry in data if 
              sigmoid(entry[2],w) < 0.5 and entry[1]==0)
    accuracy = (TP+TN)/(P+N)
    
    return(accuracy)

b) Train your perceptron on the training data and observe its accuracy on the development set. Start with batch size |T′|=10, learning rate α=0.01α=0.01 and 100 epochs. Experiment with different values for these three hyperparameters. Can you find a configuration which beats 70% accuracy on the development set? Report your best configuration and both the loss and accuracy it reaches on the development and test sets. (1P)

In [ ]:
def runepoch(rate, batch_size, epochs, dataset):
    N = 101
    w = np.random.normal(0,1,(N,1))
    for i in range(epochs):
        w = update(train_data, w, batch_size, rate)
    loss = get_loss(dataset, w)
    acc = accuracy(dataset, w)
    print(epochs,'\t',batch_size,'\t',rate,'\t',str(acc)[:5],'\t',loss)

In [ ]:
print('Epochs\t BSize\t Rate\t Acc\t Loss')
runepoch(0.01,10,100, dev_data)
runepoch(1,10,100, dev_data)
runepoch(0.01,100,100, dev_data)
runepoch(0.01,10,1000, dev_data)


runepoch(1,10,100, test_set)


An increase of learning rate showed the best result when evaluating on teh developement set.

With a learning rate of 1, 100 epochs and batch size 10 the perceptron reached an accuracy of 0.503 and loss of 793 on the test set.

c) Create a plot similar to the one from lecture 02, slide 18, i.e. plot the loss on the training set and the development set vs. the number of training epochs. For this purpose, run your perceptron with batch size |′|=1|T′|=1 and learning rate α=0.001α=0.001 for a large number of epochs (>2500). Which number of epochs is reasonable? Why does the loss on the development set not increase over time, contrary to the figure from the lecture? (1P)

-

In [ ]:
print(1/np.exp(800))
